## Stream / Voice Expression

This notebook uses the streaming API to analyze a stream of vocal audio data.

In [ ]:
import asyncio
import traceback

from utilities import download_file, encode_audio, generate_audio_stream, print_emotions

from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

In [ ]:
filepath = download_file("https://storage.googleapis.com/hume-test-data/audio/ninth-century-laugh.mp3")

async def main():
    try:
        client = HumeStreamClient("<your-api-key>")
        burst_config = BurstConfig()
        prosody_config = ProsodyConfig()
        async with client.connect([burst_config, prosody_config]) as socket:

            for sample_number, audio_sample in enumerate(generate_audio_stream(filepath)):
                encoded_sample = encode_audio(audio_sample)
                
                # Reset connection stream context between samples
                await socket.reset_stream()

                result = await socket.send_bytes(encoded_sample)

                print(f"\nStreaming sample {sample_number + 1}")

                print("Speech prosody:")
                if "warning" in result["prosody"]:
                    print(result["prosody"]["warning"])
                else:
                    emotions = result["prosody"]["predictions"][0]["emotions"]
                    print_emotions(emotions)

                print("Vocal burst")
                if "warning" in result["burst"]:
                    print(result["burst"]["warning"])
                else:
                    emotions = result["burst"]["predictions"][0]["emotions"]
                    print_emotions(emotions)
    except Exception:
        print(traceback.format_exc())

# When running the streaming API outside of a Jupyter notebook you do not need these lines.
# Jupyter has its own async event loop, so this merges main into the Jupyter event loop.
# To run this sample in a script with asyncio you can use `asyncio.run(main())`
loop = asyncio.get_event_loop()
loop.create_task(main())